<a href="https://colab.research.google.com/github/armindocachada/working-with-pandas-series/blob/main/Create_a_Panda_DataFrame_from_an_HTML_table_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It is possible to use Panda Dataframes to scrape directly an HTML table from a URL. This can be incredibly useful and a quick way to extract useful data, when you are in a hurry.

To see how valuable it is, let's use a concrete example. Imagine that we want to scrape some information about all the GPUs that Nvidia launched recently.

Wikipedia here comes to the rescue as we have fairly accurate information available in this url:

https://en.wikipedia.org/wiki/List_of_Nvidia_graphics_processing_units

We want to extract the table below:
![wikipedia_nvidia_gpu_table_screenshot.png](https://github.com/armindocachada/working-with-pandas-series/raw/main/wikipedia_nvidia_gpu_table_screenshot_v2.png)


Let's first import the pandas library

In [3]:
import pandas

# Checking the Python docs for pandas.read_html

Before we try reading an HTML table, let's check the documentation. Here is a quick way to do that directly from Python.

In [4]:
help(pandas.read_html)

Help on function read_html in module pandas.io.html:

read_html(io:Union[str, pathlib.Path, IO[~AnyStr]], match:Union[str, Pattern[~AnyStr]]='.+', flavor:Union[str, NoneType]=None, header:Union[int, Sequence[int], NoneType]=None, index_col:Union[int, Sequence[int], NoneType]=None, skiprows:Union[int, Sequence[int], slice, NoneType]=None, attrs:Union[Dict[str, str], NoneType]=None, parse_dates:bool=False, thousands:Union[str, NoneType]=',', encoding:Union[str, NoneType]=None, decimal:str='.', converters:Union[Dict, NoneType]=None, na_values=None, keep_default_na:bool=True, displayed_only:bool=True) -> List[pandas.core.frame.DataFrame]
    Read HTML tables into a ``list`` of ``DataFrame`` objects.
    
    Parameters
    ----------
    io : str, path object or file-like object
        A URL, a file-like object, or a raw string containing HTML. Note that
        lxml only accepts the http, ftp and file url protocols. If you have a
        URL that starts with ``'https'`` you might try rem

The important parameters are:



*  **str** - a url to the website with the table you want to extract, or just the HTML of the page as a string.
*  **match** - a regular expression that identifies the table that you want to extract
* **flavor** - this picks the engine doing the scraping. **bs4** stands for BeautifulSoup.

Let's try to import a table with pandas.read_html:


In [5]:
tables =pandas.read_html("https://en.wikipedia.org/wiki/List_of_Nvidia_graphics_processing_units")
display(tables)

[              Model             Launch  ...  API support             
               Model             Launch  ...     Direct3D       OpenGL
 0          STG-2000       May 22, 1995  ...          NaN          NaN
 1          Riva 128    August 25, 1997  ...          5.0          1.0
 2        Riva 128ZX  February 23, 1998  ...          5.0          1.0
 3          Riva TNT      June 15, 1998  ...          6.0          1.2
 4             Vanta     March 22, 1999  ...          6.0          1.2
 5          Vanta LT         March 2000  ...          6.0          1.2
 6     Riva TNT2 M64       October 1999  ...          6.0          1.2
 7         Riva TNT2     March 15, 1999  ...          6.0          1.2
 8     Riva TNT2 Pro   October 12, 1999  ...          6.0          1.2
 9   Riva TNT2 Ultra     March 15, 1999  ...          6.0          1.2
 10            Model             Launch  ...     Direct3D       OpenGL
 11            Model             Launch  ...  API support  API support
 
 [12

# What happened there?

We only passed an argument, the **url** from which to extract a table. With that information alone **pandas.read_html** will return you a list of all the tables it was able to extract from the page. 

**pandas.read_html()** returns a list of dataframes, one for each table it finds in the page.

It seems to have a lot of tables in this wikipedia page. Let's count how many.

In [7]:
print(len(tables))

97


That's far too many tables for what we need. In this case we only want to extract the table which contains information about the NVIDIA RTX 30 series graphics cards. 

Upon inspection the HTML of the table shows that there are no unique id or css classes that we can make use of. However that shouldn't stop us.
if we pass in the parameter **match** we should be able to pass in a string with a distinct string in the table so it can find the table that I want. 
I will pick a string which shows the CUDA **core config** for the RTX3090. This should allow me to only get the table I want.

In [62]:
pd = pandas.read_html("https://en.wikipedia.org/wiki/List_of_Nvidia_graphics_processing_units", match="10496:328:112:328:82")

print(len(pd))


1


In [63]:
display(pd[0])

Model  ... Release price (USD)
                      Model  ...    Founders Edition
0     GeForce RTX 3060[121]  ...                $329
1  GeForce RTX 3060 Ti[122]  ...                $399
2     GeForce RTX 3070[123]  ...                $499
3     GeForce RTX 3080[126]  ...                $699
4     GeForce RTX 3090[127]  ...               $1499

[5 rows x 29 columns]

In [64]:
pd[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 29 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   (Model, Model)                                                   5 non-null      object 
 1   (Launch, Launch)                                                 5 non-null      object 
 2   (Code name, Code name)                                           5 non-null      object 
 3   (Process, Process)                                               5 non-null      object 
 4   (Transistors (billion), Transistors (billion))                   5 non-null      float64
 5   (Die size (mm2), Die size (mm2))                                 5 non-null      float64
 6   (Core config[a], Core config[a])                                 5 non-null      object 
 7   (Bus interface, Bus interface)                  

Notice that we have two levels of column names. This is because the original table has two headings for the column names. We don't really need it, even though panda data frames can handle it. Let's just keep the last level

In [65]:
df = pd[0]
df.columns = df.columns.get_level_values(1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 29 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Model                               5 non-null      object 
 1   Launch                              5 non-null      object 
 2   Code name                           5 non-null      object 
 3   Process                             5 non-null      object 
 4   Transistors (billion)               5 non-null      float64
 5   Die size (mm2)                      5 non-null      float64
 6   Core config[a]                      5 non-null      object 
 7   Bus interface                       5 non-null      object 
 8   L2 Cache(MB)                        5 non-null      int64  
 9   Base core clock (MHz)               5 non-null      int64  
 10  Boost core clock (MHz)              5 non-null      int64  
 11  Memory (MT/s)                       5 non-null   

Now it is a lot easier to use the column names as we only have one level of columns.

In [66]:
df

,Model,Launch,Code name,Process,Transistors (billion),Die size (mm2),Core config[a],Bus interface,L2 Cache(MB),Base core clock (MHz),Boost core clock (MHz),Memory (MT/s),Size (GiB),Bandwidth (GB/s),Bus type,Bus width (bit),Pixel (GP/s),Texture (GT/s),Single precision,Double precision,Half precision,Tensor compute (FP16) (2:1 sparse),Rays/s (Billions),RTX OPS/s (Trillions),Ray Perf TFLOPS,TDP (Watts),NVLink support,MSRP,Founders Edition
0,GeForce RTX 3060[121],"February 25, 2021",GA106-300-A1,Samsung8N(10LP++++),13.25,300.0,3584:112:64:112:28(28) (?),PCIe 4.0 x16,3,1320,1777,15000,12,360.0,GDDR6,192,NaN,NaN,9.46 12.74,0.148 0.199,9.46 12.74,NaN,NaN,NaN,25,170,No,NaN,$329
1,GeForce RTX 3060 Ti[122],"December 2, 2020",GA104-200-A1,Samsung8N(10LP++++),17.40,392.5,4864:152:80:152:38(38) (6),PCIe 4.0 x16,4,1410,1665,14000,8,448.0,GDDR6,256,NaN,NaN,13.7 16.20,0.214 0.253,13.7 16.20,? 129.6,NaN,NaN,NaN,200,No,NaN,$399
2,GeForce RTX 3070[123],"October 29, 2020[124]",GA104-300-A1,Samsung8N(10LP++++),17.40,392.5,5888:184:96:184:46(46) (6),PCIe 4.0 x16,4,1500,1725,14000,8,448.0,GDDR6,256,96.0 110.72,276.0 318.32,17.66 20.37,0.276 0.318,17.66 20.37,141.31 162.98,NaN,NaN,40[125],220,No,NaN,$499
3,GeForce RTX 3080[126],"September 17, 2020",GA102-200-KD-A1,Samsung8N(10LP++++),28.30,628.4,8704:272:96:272:68(68) (7),PCIe 4.0 x16,5,1440,1710,19000,10,760.0,GDDR6X,320,126.72 150.48,391.68 465.12,25.06 29.76,0.392 0.465,25.06 29.76,200.54 238.14,NaN,NaN,58[125],320,No,NaN,$699
4,GeForce RTX 3090[127],"September 24, 2020",GA102-300-A1,Samsung8N(10LP++++),28.30,628.4,10496:328:112:328:82(82) (7),PCIe 4.0 x16,6,1395,1695,19500,24,935.8,GDDR6X,384,134.4 162.72,459.20 555.96,29.38 35.68,0.459 0.558,29.38 35.68,235.08 285.48,NaN,NaN,69[125],350,2-way NVLink,NaN,$1499


Perfect! We got the table we wanted. Time to do some data clenup. Notice that the "Launch date" has an unwanted string[24] at the end. Let's remove it

In [67]:
df["Launch"]

0        February 25, 2021
1         December 2, 2020
2    October 29, 2020[124]
3       September 17, 2020
4       September 24, 2020
Name: Launch, dtype: object

Let's get rid of it using a regular expression.

In [68]:
df['Launch'] = df['Launch'].str.replace(r"\[\d*?\]","")

In [56]:
df

,Model,Launch,Code name,Process,Transistors (billion),Die size (mm2),Core config[a],Bus interface,L2 Cache(MB),Base core clock (MHz),Boost core clock (MHz),Memory (MT/s),Size (GiB),Bandwidth (GB/s),Bus type,Bus width (bit),Pixel (GP/s),Texture (GT/s),Single precision,Double precision,Half precision,Tensor compute (FP16) (2:1 sparse),Rays/s (Billions),RTX OPS/s (Trillions),Ray Perf TFLOPS,TDP (Watts),NVLink support,MSRP,Founders Edition
0,GeForce RTX 3060[121],"February 25, 2021",GA106-300-A1,Samsung8N(10LP++++),13.25,300.0,3584:112:64:112:28(28) (?),PCIe 4.0 x16,3,1320,1777,15000,12,360.0,GDDR6,192,NaN,NaN,9.46 12.74,0.148 0.199,9.46 12.74,NaN,NaN,NaN,25,170,No,NaN,$329
1,GeForce RTX 3060 Ti[122],"December 2, 2020",GA104-200-A1,Samsung8N(10LP++++),17.40,392.5,4864:152:80:152:38(38) (6),PCIe 4.0 x16,4,1410,1665,14000,8,448.0,GDDR6,256,NaN,NaN,13.7 16.20,0.214 0.253,13.7 16.20,? 129.6,NaN,NaN,NaN,200,No,NaN,$399
2,GeForce RTX 3070[123],"October 29, 2020",GA104-300-A1,Samsung8N(10LP++++),17.40,392.5,5888:184:96:184:46(46) (6),PCIe 4.0 x16,4,1500,1725,14000,8,448.0,GDDR6,256,96.0 110.72,276.0 318.32,17.66 20.37,0.276 0.318,17.66 20.37,141.31 162.98,NaN,NaN,40[125],220,No,NaN,$499
3,GeForce RTX 3080[126],"September 17, 2020",GA102-200-KD-A1,Samsung8N(10LP++++),28.30,628.4,8704:272:96:272:68(68) (7),PCIe 4.0 x16,5,1440,1710,19000,10,760.0,GDDR6X,320,126.72 150.48,391.68 465.12,25.06 29.76,0.392 0.465,25.06 29.76,200.54 238.14,NaN,NaN,58[125],320,No,NaN,$699
4,GeForce RTX 3090[127],"September 24, 2020",GA102-300-A1,Samsung8N(10LP++++),28.30,628.4,10496:328:112:328:82(82) (7),PCIe 4.0 x16,6,1395,1695,19500,24,935.8,GDDR6X,384,134.4 162.72,459.20 555.96,29.38 35.68,0.459 0.558,29.38 35.68,235.08 285.48,NaN,NaN,69[125],350,2-way NVLink,NaN,$1499


Lets convert the launch date to a timestamp now

In [59]:
df['Launch'] = pandas.to_datetime(df['Launch'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 29 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Model                               5 non-null      object        
 1   Launch                              5 non-null      datetime64[ns]
 2   Code name                           5 non-null      object        
 3   Process                             5 non-null      object        
 4   Transistors (billion)               5 non-null      float64       
 5   Die size (mm2)                      5 non-null      float64       
 6   Core config[a]                      5 non-null      object        
 7   Bus interface                       5 non-null      object        
 8   L2 Cache(MB)                        5 non-null      int64         
 9   Base core clock (MHz)               5 non-null      int64         
 10  Boost core clock (MHz)        

Having a valid datetime column will help with plotting graphs.

In [35]:
#October 29, 2020[124]
def customDateParser(x):
  dateStr = x.replace("[24]",""); 
  return pd.datetime.strptime(dateStr, "%M %d, %Y")



pd = pandas.read_html("https://en.wikipedia.org/wiki/List_of_Nvidia_graphics_processing_units", match="10496:328:112:328:82")

In [30]:
pd[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 29 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   (Model, Model)                                                   5 non-null      object 
 1   (Launch, Launch)                                                 5 non-null      object 
 2   (Code name, Code name)                                           5 non-null      object 
 3   (Process, Process)                                               5 non-null      object 
 4   (Transistors (billion), Transistors (billion))                   5 non-null      float64
 5   (Die size (mm2), Die size (mm2))                                 5 non-null      float64
 6   (Core config[a], Core config[a])                                 5 non-null      object 
 7   (Bus interface, Bus interface)                  

In [36]:
pd[0]["Launch"]

,Launch
0,"February 25, 2021"
1,"December 2, 2020"
2,"October 29, 2020[124]"
3,"September 17, 2020"
4,"September 24, 2020"
